# <center> Get user graph for `Russian`

In [17]:
import sys
import util
import json
import time
from twython import Twython

### Twitter API credentials

In [18]:
APP_KEY = "AV4KujYNS1y6HdmQ4zXxHw2Tx"
APP_SECRET = "xd2FuFaS5zE1ON407C3U5X0kdEtRJIjH9OMDmgyMmxQCZMFlli"
OAUTH_TOKEN = "1058011325316636672-HkpzhyiOHdJkuwi3FD9QrkMFbuHuVt"
OAUTH_TOKEN_SECRET = "iiD75H9ERtkATnd74pVRBfM7TI189BSmDasYzN2uUO123"

### Retrieve user graph

Function to get set of followers from user id.

In [31]:
def get_followers(uid):
    
    # init
    twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    followers_set = set()
    cursor = -1
    
    # while there are followers left for this user
    while cursor != 0:
        retry = True
        
        # if we haven't this page of followers yet
        while retry:
            try:
                result = twitter.get_followers_ids(id=uid, cursor=cursor)
                followers_set = followers_set.union(result['ids'])
                cursor = result['next_cursor']
                retry = False
                
            # twitter api rate limit exceeded --> sleep then reconnect and retry
            except:
                reset_time = float(twitter.get_lastfunction_header('x-rate-limit-reset'))
                del twitter
                while time.time() <= reset_time:
                    time_left = reset_time - time.time()
                    sys.stdout.flush()
                    sys.stdout.write("Sleeping... {:.0f}min {:.0f}sec left\r".format(time_left//60, time_left%60))
                    time.sleep(1)
                twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
                retry = True
                
    # return set of followers
    return followers_set

Get user set.

In [20]:
users = set(util.get_authors("../Datasets/russian_rtid.txt").keys())

Get the followgraph of users present in the trace.

In [32]:
FollowGraph = dict()

for u in users:
    FollowGraph[u] = get_followers(u).intersection(users)

KeyboardInterrupt: 

In [25]:
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
twitter.get_followers_ids(screen_name="villedelyon")
float(twitter.get_lastfunction_header('x-rate-limit-reset'))

1560266008.0

In [27]:
(1560266008.0 - time.time())//60

13.0

### Write adj list to out

In [ ]:
out = open("../Datasets/russian_adjList.txt", "w")

for u in FollowGraph:
    for v in FollowGraph[u]:
        out.write("{} {}\n".format(u,v))
        
out.close()       

## Old

In [ ]:
FollowGraph = dict()

for line in open("../Datasets/russian_rtid.txt"):
    line = line.split()
    uid, rtid = int(line[2]), int(line[3])
    
    # get followgraph for uid
    if uid not in FollowGraph:
        FollowGraph[uid] = get_followers(uid).intersection(users)
    
    # get followgraph for rtid
    if rtid != -1 and rtid in Author:
        rtu = Author[rtid]
        if rtu not in FollowGraph:
            FollowGraph[rtu] = get_followers(uid).intersection(users)